In [1]:
import gc
import os
import pickle
import sys
import time
import uuid
from typing import Final, Optional

import cv2 as OpenCV
import numpy as np
from matplotlib import pyplot as plt
from numpy.linalg import norm
from scipy.cluster.vq import kmeans, vq
from scipy.spatial import Delaunay

In [2]:
class Image:
    def __init__(self, img_id, rgb_image, gray_image, mask, keypoints, descriptors, path):
        self.img_id: int = int(img_id)
        self.unique_id: uuid = uuid.uuid4()
        self.rgb_image: Image = rgb_image
        self.gray_image: Image = gray_image
        self.keypoints: list[OpenCV.KeyPoint] = keypoints
        self.descriptors: np.ndarray = descriptors
        self.path: str = path
        self.mask: Image = mask

    @property
    def length(self):
        return f"{len(self.keypoints)}" if len(self.keypoints) == len(self.descriptors) else f"{len(self.keypoints)}, {len(self.descriptors)}"
    
    def draw_sift_features(self):
        image_with_sift = OpenCV.drawKeypoints(self.rgb_image, self.keypoints, None, flags=OpenCV.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        plt.imshow(image_with_sift)
        plt.title("Image with SIFT Features")
        plt.axis('off')
        plt.show()

    def display_rgb_image(self, title: Optional[str] = None):
        image = self.rgb_image
        plt.imshow(image)
        if title is not None:
            plt.title(title)
        plt.axis('off')
        plt.show()

    def display_gray_image(self, title: Optional[str] = None):
        image = self.gray_image
        plt.gray()
        plt.imshow(image)
        if title is not None:
            plt.title(title)
        plt.axes('off')
        plt.show()
        
    def display_mask_image(self, title: Optional[str] = None):
        image = self.mask
        plt.gray()
        plt.imshow(image)
        if title is not None:
            plt.title(title)
        # plt.axes('off')
        plt.show()
        
    def display_dialated_image(self, title: Optional[str] = None):
        print(self.mask.shape)
        print(self.rgb_image.shape)
        image = OpenCV.bitwise_and(self.rgb_image, self.rgb_image, mask=self.mask)
        plt.imshow(image)
        if title is not None:
            plt.title(title)
        # plt.axis('off')
        plt.show()
        
    def __repr__(self):
        return f"Image({self.img_id})"
    
    def __str__(self):
        return self.__repr__()
    
    def __eq__(self, other):
        return self.unique_id == other.unique_id
    
    def __hash__(self):
        return hash(self.img_id)
    
    def __getstate__(self):
        state = self.__dict__.copy()
        state['keypoints'] = [tuple(k.pt) + (k.size, k.angle, k.response, k.octave, k.class_id) for k in self.keypoints]
        return state
    
    def __setstate__(self, state):
        state['keypoints'] = [OpenCV.KeyPoint(x, y, size, angle, response, octave, class_id) for x, y, size, angle, response, octave, class_id in state['keypoints']]
        self.__dict__ = state

class FeatureMatches:
    def __init__(self, image_one: Image, image_two: Image, matches: list[OpenCV.DMatch]):
        self.image_one: Image = image_one
        self.image_two: Image = image_two
        self.matches: list[OpenCV.DMatch] = matches

    def draw_matches(self, output_filename: str) -> None:
        combined_image = OpenCV.hconcat([
            self.image_one.rgb_image,
            self.image_two.rgb_image
        ])

        for match in self.matches:
            x1, y1 = self.image_one.keypoints[match.queryIdx].pt
            x2, y2 = self.image_two.keypoints[match.trainIdx].pt
            # Draw a line connecting the matched keypoints
            OpenCV.line(
                combined_image, 
                (int(x1), int(y1)), 
                (int(x2) + self.image_one.rgb_image.shape[1], int(y2)), 
                (0, 255, 0), 
                1
            )

        OpenCV.imwrite(output_filename, combined_image)
        
    def simulate_matches(self, output_dir: str) -> None:
        combined_image = OpenCV.hconcat([
            self.image_one.rgb_image,
            self.image_two.rgb_image
        ])

        for i, match in enumerate(self.matches):
            x1, y1 = self.image_one.keypoints[match.queryIdx].pt
            x2, y2 = self.image_two.keypoints[match.trainIdx].pt
            # Draw a line connecting the matched keypoints
            OpenCV.line(
                combined_image, 
                (int(x1), int(y1)), 
                (int(x2) + self.image_one.rgb_image.shape[1], int(y2)), 
                (0, 255, 0), 
                1
            )
            
            resized_image = OpenCV.resize(combined_image, None, fx=0.5, fy=0.5, interpolation=OpenCV.INTER_AREA)
            OpenCV.imwrite(f"{output_dir}/{i}.jpg", resized_image)

    def animate_matches(self, output_dir: str, scale_factor: float = 0.5) -> None:
        import imageio
        combined_image = OpenCV.hconcat([
            self.image_one.rgb_image,
            self.image_two.rgb_image
        ])

        # Create a GIF writer
        with imageio.get_writer(f"{output_dir}/animated_matches.gif", mode='I', duration=0.001) as writer:
            for i, match in enumerate(self.matches):
                x1, y1 = self.image_one.keypoints[match.queryIdx].pt
                x2, y2 = self.image_two.keypoints[match.trainIdx].pt
                # Draw a line connecting the matched keypoints
                OpenCV.line(
                    combined_image, 
                    (int(x1), int(y1)), 
                    (int(x2) + self.image_one.rgb_image.shape[1], int(y2)), 
                    (0, 255, 0), 
                    1
                )
                # Resize the image
                resized_image = OpenCV.resize(combined_image, None, fx=scale_factor, fy=scale_factor, interpolation=OpenCV.INTER_AREA)
                # Convert the image from OpenCV BGR format to RGB format
                rgb_image = OpenCV.cvtColor(resized_image, OpenCV.COLOR_BGR2RGB)
                # Append the frame to the GIF
                writer.append_data(rgb_image)

    def __repr__(self):
        return f"FeatureMatches({self.image_one}, {self.image_two} ---> {len(self.matches)})"

    def __getstate__(self):
        state = self.__dict__.copy()
        state['matches'] = [
            {'queryIdx': m.queryIdx, 'trainIdx': m.trainIdx, 'distance': m.distance} for m in self.matches
        ]
        return state
    
    def __setstate__(self, state):
        state['matches'] = [
            OpenCV.DMatch(match['queryIdx'], match['trainIdx'], match['distance']) for match in state['matches']
        ]
        self.__dict__ = state
    
class Images:
    def __init__(self, images: list[Image], image_set_name: str):
        self.id = uuid.uuid4()
        self.images: list[Image] = images
        self.image_set_name: str = image_set_name
        self.feature_matches: list[FeatureMatches] = []
        self.similar_images: dict[list[Image]] = {}
        self.num_clusters: int = 50

    def save_feature_matches(self):
        for match in self.feature_matches:
            match.draw_matches(f"data/hammer/output/feature-match/{match.image_one.img_id}_{match.image_two.img_id}.jpg")

    def __len__(self):
        return len(self.images)
    
    def display_similar_images(self, key):
        print(f"cluster {key}")
        print("-----------------------------------------------------")
        for value in self.similar_images[key]:
            print(value)
            rgb_image = OpenCV.cvtColor(OpenCV.imread(value.path), OpenCV.COLOR_BGR2RGB)
            plt.imshow(rgb_image)
            plt.title(value.path)
            plt.axis('off')
            plt.show()

    def save_similar_images(self):
        for cluster in self.similar_images.keys():
            if not os.path.exists(f"data/{self.image_set_name}/output/image-match/{cluster}"):
                os.makedirs(f"data/{self.image_set_name}/output/image-match/{cluster}")
            for value in self.similar_images[cluster]:
                OpenCV.imwrite(f"data/{self.image_set_name}/output/image-match/{cluster}/{value.img_id}.jpg", value.rgb_image)

    def __getitem__(self, key: int) -> Image:
        for image in self.images:
            if image.img_id == key:
                return image
        raise KeyError(f'Image with img_id {key} not found.')

In [3]:
images = Images([], "hammer")

In [4]:
image_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\images\1.jpg'
mask_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\masks\1.jpg'
rgb_image = OpenCV.cvtColor(OpenCV.imread(image_path), OpenCV.COLOR_BGR2RGB)
gray_image = OpenCV.cvtColor(rgb_image, OpenCV.COLOR_RGB2GRAY)
mask = OpenCV.imread(mask_path, OpenCV.IMREAD_GRAYSCALE)
kernel = np.ones((5, 5), np.uint8)
dilated_mask = OpenCV.dilate(mask, kernel, iterations=20)
images.images.append(Image(1, rgb_image, gray_image, dilated_mask, [], [], image_path))

In [5]:
image_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\images\2.jpg'
mask_path = r'C:\Users\yousf\OneDrive\Desktop\University\Graduation Project\Codes\photogrammetry\src\data\hammer\masks\2.jpg'
rgb_image = OpenCV.cvtColor(OpenCV.imread(image_path), OpenCV.COLOR_BGR2RGB)
gray_image = OpenCV.cvtColor(rgb_image, OpenCV.COLOR_RGB2GRAY)
mask = OpenCV.imread(mask_path, OpenCV.IMREAD_GRAYSCALE)
dilated_mask = OpenCV.dilate(mask, kernel, iterations=20)
images.images.append(Image(2, rgb_image, gray_image, dilated_mask, [], [], image_path))

In [6]:
def compute_keypoints_descriptors(images: list[Image], SIFT: OpenCV.SIFT) -> None:
    for img in images.images:
        keypoints: list[OpenCV.KeyPoint]
        descriptors: np.ndarray
        dialated_image = OpenCV.bitwise_and(img.gray_image, img.gray_image, mask=img.mask)
        keypoints, descriptors = SIFT.detectAndCompute(dialated_image, None)
        img.keypoints = keypoints
        img.descriptors = descriptors

In [7]:
sift = OpenCV.SIFT_create(contrastThreshold=0.01)
compute_keypoints_descriptors(images, sift)

In [8]:
def feature_matching(
        img_one_descriptors: np.ndarray, 
        img_two_descriptors: np.ndarray,
        checker: bool = False
    ) -> list[OpenCV.DMatch]:
    if checker:
        matcher = OpenCV.BFMatcher(crossCheck=True)
        return matcher.match(img_one_descriptors, img_two_descriptors)
    else:
        matcher = OpenCV.BFMatcher()
        return matcher.match(img_one_descriptors, img_two_descriptors)

def apply_ransac(matches, keypoints1, keypoints2, threshold = 3.0):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    _, mask = OpenCV.findHomography(src_pts, dst_pts, OpenCV.RANSAC, threshold)
    matches_mask = mask.ravel().tolist()
    return [m for m, keep in zip(matches, matches_mask) if keep]

def data_feature_matching(images: Images) -> None:
    import itertools
    for key, values in images.similar_images.items():
        for image, matched_image in itertools.combinations(values, 2):
            feature_matching_output = feature_matching(image.descriptors, matched_image.descriptors)
            ransac_output = apply_ransac(feature_matching_output, image.keypoints, matched_image.keypoints, threshold=150)
            images.feature_matches.append(FeatureMatches(image, matched_image, ransac_output))

In [9]:
feature_matching_output = feature_matching(images.images[0].descriptors, images.images[1].descriptors)
feature_matcher = FeatureMatches(images.images[0], images.images[1], feature_matching_output)
feature_matcher.draw_matches("data/hammer/testing_outputs/1_2_Virgin.jpg")
print(len(feature_matcher.matches))

24756


In [10]:
feature_matching_output = feature_matching(images.images[0].descriptors, images.images[1].descriptors, True)
feature_matcher = FeatureMatches(images.images[0], images.images[1], feature_matching_output)
feature_matcher.draw_matches("data/hammer/testing_outputs/1_2_Check.jpg")
print(len(feature_matcher.matches))

10340


In [11]:
feature_matching_output = feature_matching(images.images[0].descriptors, images.images[1].descriptors, True)
ransac_output = apply_ransac(feature_matching_output, images.images[0].keypoints, images.images[1].keypoints, threshold=150)
feature_matcher = FeatureMatches(images.images[0], images.images[1], ransac_output)
feature_matcher.draw_matches("data/hammer/testing_outputs/1_2_ransac.jpg")
print(len(feature_matcher.matches))

7362


In [24]:
# feature_matcher.simulate_matches("data/hammer/testing_feature_match")

In [12]:
feature_matcher.simulate_matches("data/hammer/testing_feature_match")

In [ ]:
def generate_point_cloud(images: Images, K_matrix: np.ndarray, **kwargs) -> np.ndarray:
    point_cloud = []

    for feature_match in images.feature_matches:
        image_one = feature_match.image_one
        image_two = feature_match.image_two

        # Extract matched keypoints
        keypoints_one = np.array([image_one.keypoints[m.queryIdx].pt for m in feature_match.matches])
        keypoints_two = np.array([image_two.keypoints[m.trainIdx].pt for m in feature_match.matches])

        # Estimate the essential matrix
        E, mask = OpenCV.findEssentialMat(keypoints_one, keypoints_two, K_matrix, method=OpenCV.RANSAC, prob=0.999, threshold=1.0)
        _, R, t, _ = OpenCV.recoverPose(E, keypoints_one, keypoints_two, K_matrix)

        # Create projection matrices
        P1 = K_matrix @ np.hstack((np.eye(3), np.zeros((3, 1))))
        P2 = K_matrix @ np.hstack((R, t))

        # Triangulate points
        points_4D = OpenCV.triangulatePoints(P1, P2, keypoints_one.T, keypoints_two.T)
        points_3D = (points_4D / points_4D[3])[:3]

        point_cloud.append(points_3D)

    # Merge all point clouds into one
    point_cloud = np.hstack(point_cloud).T